<a href="https://colab.research.google.com/github/nikhilno1/indoor-location/blob/master/indoor_location_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.sparse
import pickle
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer

In [0]:
df=pd.read_csv("https://raw.githubusercontent.com/nikhilno1/indoor-location/master/sample_data.csv", sep=',')    

In [9]:
# Create training and validation sets
sz = df.shape
train_df = df.iloc[:int(sz[0] * 0.7), :]
test_df = df.iloc[int(sz[0] * 0.7):, :]

# Keep only the srs columns
train_df1 = train_df.iloc[:, :32]

# Separate X & Y for training
train_X = train_df1.values
train_Y1 = train_df.iloc[:, 32].values
train_Y2 = train_df.iloc[:, 33].values

test_df1 = test_df.iloc[:, :32]

# Separate X & Y for test
test_X = test_df1.values
test_Y1 = test_df.iloc[:, 32].values
test_Y2 = test_df.iloc[:, 33].values

classes = df.ue_loc_1.unique()
num_classes = df.ue_loc_1.nunique()

# integer encode
le = LabelEncoder()
train_Y = le.fit_transform(train_Y1.tolist())
test_Y = le.fit_transform(test_Y1.tolist())

xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)

# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = num_classes

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist)
#bst.dump_model('dump.nice.txt', 'myfeatmap.txt')
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
print('Test error using softmax = {}'.format(error_rate))

# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bst.predict(xg_test).reshape(test_Y.shape[0], num_classes)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != test_Y) / test_Y.shape[0]
print('Test error using softprob = {}'.format(error_rate))

# # Serialize both the pipeline and binarizer to disk.
# with open('my_sklearn_objects.pkl', 'wb') as f:
#     pickle.dump((bst, le), f)

[0]	train-merror:0.001488	test-merror:0.00284
[1]	train-merror:0.000812	test-merror:0.00284
[2]	train-merror:0.001488	test-merror:0.00284
[3]	train-merror:0.000812	test-merror:0.00284
[4]	train-merror:0.000812	test-merror:0.00284
Test error using softmax = 0.002840012622278321
[0]	train-merror:0.001488	test-merror:0.00284
[1]	train-merror:0.000812	test-merror:0.00284
[2]	train-merror:0.001488	test-merror:0.00284
[3]	train-merror:0.000812	test-merror:0.00284
[4]	train-merror:0.000812	test-merror:0.00284
Test error using softprob = 0.002840012622278321


### Load Model

In [10]:
# Hydrate the serialized objects.
#with open('my_sklearn_objects.pkl', 'rb') as f:
#    bst1, le1 = pickle.load(f)
    
data = [100.0,99.0,124,128,95,190,98,92,158,96,97,141,99,97,99,151,98,96,99,100,99,94,124,148,98,98,120,100,95,99,98.0,100]
data = [float(val) for val in data]
dtest = xgb.DMatrix(data)
test_pred = bst.predict(dtest)
test_label = np.argmax(test_pred, axis=1)
final_pred = le.inverse_transform(test_label)
final_pred[0]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


'rp6'